# Demo

## Load Environment Vars

In [4]:
from dotenv import load_dotenv
load_dotenv('./.env')

True

## Load Examples

In [5]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)


## Create Selector and Examples Prompt

In [6]:
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    # The list of examples available to select from.
    examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity.
    embeddings=OpenAIEmbeddings(),
    # The VectorStore class that is used to store the embeddings and do a similarity search over.
    vectorstore_cls=FAISS,
    # The number of examples to produce.
    k=2,
)
mmr_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

In [7]:
# Input is a feeling, so should select the happy/sad example as the first one
print(mmr_prompt.format(adjective="worried"))

Give the antonym of every input

Input: happy
Output: sad

Input: windy
Output: calm

Input: worried
Output:


## Data Models

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field

class ProjectInput(BaseModel):
    """The project data defined by the client (may need changes)."""
    
    subcategory: int = Field(description="The subcategory of the project.")
    title: str = Field(description="The title of the project.")
    description: str = Field(description="The description of the project.")
    abilities: list[str] = Field(description="The abilities desired for the project.")
    
class ProjectPrediction(BaseModel):
    """The predicted project data based on the input. Some fields might be left undefined depending on the project being approved or not."""
    
    approve: bool = Field(description="Whether the project should be approved or not.")
    subcategory: int | None = Field(description="The subcategory of the project (when approved).")
    title: str | None = Field(description="The title of the project (when approved).", ge=25, le=75)
    description: str | None = Field(description="The description of the project (when approved).", ge=50, le=2000)
    reprovation_reason: int | None = Field(description="The reason for the project to be reproved (when reproved).")
    reprovation_comment: str | None = Field(description="The comment for the reprovation (when reproved).")
    
class ProjectInfo(BaseModel):
    """Additional information about the project."""
    
    reasoning: str = Field(description="The reasoning behind the prediction.")
    confidence: float = Field(description="The confidence of the prediction.", ge=0, le=1)
    bot_generated: bool = Field(description="How likely is for the project to be bot generated.", ge=0, le=1)
    
    
class ProjectOutput(BaseModel):
    """The predicted project data based on the input, along with additional information."""
    
    prediction: ProjectPrediction = Field(description="The predicted project data based on the input.")
    info: ProjectInfo = Field(description="Additional information about the project.")

## Stores

In [ ]:
def get_full_example(idx):
    input = ProjectInput(
        subcategory=1,
        title=f"PRECISO DE DESIGNER GRFICO - #{idx + 1}",
        description=f"Preciso de desgner grafico para criação de logotipo\nMeu telefone é 98765-4321",
        abilities=["Habilidade 1", "Habilidade 2", "Habilidade 3"],
    )
    prediction = ProjectPrediction(
        subcategory=1,
        title=f"Designer gráfico para criação de Logotipo - #{idx + 1}",
        description=f"Preciso de designer gráfico para criação de logotipo.",
    )
    info = ProjectInfo(
        reasoning="Alteração do título para ficar mais conciso e ajustes em sua capitalização. Correções ortográficas na descrição. Remoção de telefone de contato.",
        confidence=0.8,
        bot_generated=0.4,
    )
    output = ProjectOutput(prediction=prediction, info=info)
    return input, output

def get_example(idx):
    input, output = get_full_example(idx)
    prediction = output.prediction
    return input, prediction

# Examples of a pretend task of creating antonyms.
examples = [get_example(i) for i in range(1000)]

In [ ]:
from langchain.storage import InMemoryByteStore

store = InMemoryByteStore()

store.mset([("k1", b"v1"), ("k2", b"v2")])
print(store.mget(["k1", "k2"]))